In [719]:
import pandas as pd
import matplotlib.pyplot as plt
from collect import get_data
import numpy as np
import re

In [720]:
get_data()

You are Successfully connected to -  ('PostgreSQL 9.6.16 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit',) 

data updated


''

In [784]:
df_stock = pd.read_csv('Data/stock.csv')
df_price = pd.read_csv('Data/price.csv')
df_remove = pd.read_csv('Data/to_remove.csv')

In [785]:
df_stock.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,first_main_system_id,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1649441,37707.47966,CTN#ANGAZA
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1308543,37707.47966,CTN#ANGAZA
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1219948,37707.00000,CTN#ANGAZA
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1315845,37707.00000,CTN#ANGAZA
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1214642,37707.47966,CTN#ANGAZA


In [786]:
df_stock.dtypes

product_id                int64
created_at               object
serial                   object
product                  object
etat                     object
prefix                  float64
true_serial              object
stock_type               object
region                   object
is_main_system             bool
main_system_id          float64
first_main_system_id    float64
stock_name               object
systems                  object
product_type             object
category                 object
country                  object
is_archived                bool
serial_number            object
cost_price              float64
carton_id                object
dtype: object

In [787]:
print(f'le shape du dataframe {df_stock.shape}')
df_stock.isnull().sum()

le shape du dataframe (77038, 21)


product_id                  0
created_at                  0
serial                      0
product                     0
etat                        0
prefix                  72278
true_serial                 0
stock_type                  0
region                      0
is_main_system              0
main_system_id          77026
first_main_system_id    47544
stock_name                  0
systems                 71844
product_type                0
category                 2229
country                     0
is_archived                 0
serial_number           55077
cost_price              57914
carton_id               60306
dtype: int64

In [788]:
df_stock = df_stock[~df_stock['true_serial'].isnull()]
df_stock.shape

(77038, 21)

In [789]:
print(f'Nombre de serial en stock {df_stock.true_serial.nunique()}')
print(f'Verification de doublons {df_stock.true_serial.nunique()== df_stock.shape[0]}')

Nombre de serial en stock 76960
Verification de doublons False


In [790]:
df_stock['true_serial'].duplicated().sum()

78

In [791]:
df_stock.drop_duplicates(inplace=True)

In [792]:
df_stock[df_stock['true_serial'].duplicated()]

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,first_main_system_id,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id
12577,924637,2021-10-01 08:29:20.810698,933507239,WowSolar TV Screen 19,Endommagé,933.0,507239,multi_manager,BAMAKO,False,...,NaN,BAMAKO,NaN,WowSolar TV Screen 19,SHS+TV,Mali,False,NaN,NaN,NaN
18361,862079,2021-04-08 13:35:50.548475,275523970,WowSolar TV Screen 22,Endommagé,275.0,523970,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,WowSolar TV Screen 22,SHS+TV,Senegal,False,NaN,NaN,NaN
34125,754521,2021-02-19 22:34:39.551187,933506814,WowSolar TV Screen 19,Neuf,933.0,506814,personal,ENUGU,False,...,49557.0,Paschal Ihekuna - paschal.ihekuna,NaN,WowSolar TV Screen 19,SHS+TV,Nigeria,False,NaN,NaN,NaN
56783,808591,2021-02-19 23:01:08.686916,275522621,WowSolar TV Screen 22,Neuf,275.0,522621,multi_manager,BAMAKO,False,...,62464.0,ZZZ - Batteries fin de contrat,NaN,WowSolar TV Screen 22,SHS+TV,Mali,True,NaN,NaN,NaN
71140,928413,2021-11-03 13:04:37.999246,895111699,WowSolar Batterie GM 80W,waste,895.0,111699,multi_manager,FATICK,False,...,NaN,Fatick,NaN,WowSolar Batterie GM 80W,SHS+TV,Senegal,False,111699,9148.0,CTN#CLIPPERS28012020
72024,859076,2021-03-31 18:36:55.114036,275527106,WowSolar TV Screen 22,Endommagé,275.0,527106,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,WowSolar TV Screen 22,SHS+TV,Senegal,False,NaN,NaN,NaN


In [793]:
df_stock[df_stock['is_archived']==True]['stock_name'].unique()

array(['ZZZ - Batteries fin de contrat', 'YYY - Kits perdus remboursés',
       'ZZZ', 'ZZZ - Sunday Osajiokwu', 'ZZZ - Michael Sonubi',
       'ZZZ - LGS', 'ZZZ - Abimbola Omeni', 'YYY - Ongoing Reimbursement',
       'ZZZ - Monday Oliseyenum', 'ZZZ - Lenny Okoh', 'ZZZ - Titus Gesa',
       'YYY - Kits en cours de remboursement', 'XXX',
       'XXX- Erreurs de déploiement', 'ZZZ - SN', 'XXX - Sold Component',
       'ZZZ - Chinomso Ejiogu', 'ZZZ - Paschal Ihekuna',
       'YYY - Reimbursed Kits', 'XXX ML'], dtype=object)

## Systems

In [794]:
df_system = df_stock[df_stock['is_main_system']==True]
print(f'Nombre de systems compléte {df_system.shape}')
df_system.head()

Nombre de systems compléte (5194, 21)


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,first_main_system_id,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id
38,949441,2022-02-09 12:40:32.084859,798410722,Burn stove component,Neuf,798.0,410722,personal,ABIA,True,...,NaN,SIMON MENTOR AGBO - OHAFIA01,Burn Stove,Burn Stove,accessories,Nigeria,False,410722,NaN,NaN
49,13955,2017-03-16 15:29:28.000000,7321649,SKH Controller,recycle,NaN,7321649,multi_manager,KAOLACK,True,...,NaN,Kaolack,Sun King Home 60 Easy Buy,Sun King Home 60 Easy Buy,pico,Senegal,False,7321649,37707.47966,CTN#ANGAZA
98,949439,2022-02-09 12:40:32.056257,798410720,Burn stove component,Neuf,798.0,410720,personal,RIVERS,True,...,NaN,SORLESI LEBARI FAITH - BORI09,Burn Stove,Burn Stove,accessories,Nigeria,False,410720,NaN,NaN
113,16757,2017-05-05 20:49:21.000000,4405759,SKH Controller,recycle,NaN,4405759,multi_manager,KAOLACK,True,...,NaN,Kaolack,Sun King Home 60 Easy Buy,Sun King Home 60 Easy Buy,pico,Senegal,False,4405759,37707.47966,CTN#ANGAZA
130,17497,2017-05-18 14:04:47.000000,9266149,SKH Controller,recycle,NaN,9266149,multi_manager,KAOLACK,True,...,NaN,Kaolack,Sun King Home 60 Easy Buy,Sun King Home 60 Easy Buy,pico,Senegal,False,9266149,37707.47966,CTN#ANGAZA


In [795]:
df_system.isnull().sum()

product_id                 0
created_at                 0
serial                     0
product                    0
etat                       0
prefix                  3279
true_serial                0
stock_type                 0
region                     0
is_main_system             0
main_system_id          5194
first_main_system_id    5194
stock_name                 0
systems                    0
product_type               0
category                 853
country                    0
is_archived                0
serial_number             23
cost_price              1804
carton_id               2727
dtype: int64

In [796]:
# value_condition = (df_system['etat'].isin(['Endommagé','waste','canceled']) | df_system['is_archived']==True)
# loss_condition = (df_system['etat'].isin(['Endommagé','waste','canceled']) | df_system['stock_name'].str.lower().str.startswith('zzz'))

# df_system['valeur'] = np.where(value_condition,0,df_system['cost_price'])
# df_system['loss'] = np.where(loss_condition,0,df_system['cost_price'])

# df_system.head()

In [797]:
df_system['group'] = 'system'

/tmp/ipykernel_20328/1580189400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_system['group'] = 'system'


In [798]:
df_system.columns

Index(['product_id', 'created_at', 'serial', 'product', 'etat', 'prefix',
       'true_serial', 'stock_type', 'region', 'is_main_system',
       'main_system_id', 'first_main_system_id', 'stock_name', 'systems',
       'product_type', 'category', 'country', 'is_archived', 'serial_number',
       'cost_price', 'carton_id', 'group'],
      dtype='object')

In [799]:
df_system.dtypes

product_id                int64
created_at               object
serial                   object
product                  object
etat                     object
prefix                  float64
true_serial              object
stock_type               object
region                   object
is_main_system             bool
main_system_id          float64
first_main_system_id    float64
stock_name               object
systems                  object
product_type             object
category                 object
country                  object
is_archived                bool
serial_number            object
cost_price              float64
carton_id                object
group                    object
dtype: object

In [800]:
df_compo = df_stock[df_stock['is_main_system']==False]

In [801]:
df_compo.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,first_main_system_id,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1649441,37707.47966,CTN#ANGAZA
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1308543,37707.47966,CTN#ANGAZA
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,NaN,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1219948,37707.00000,CTN#ANGAZA
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1315845,37707.00000,CTN#ANGAZA
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,NaN,Fatick,NaN,SKH Controller,pico,Senegal,False,1214642,37707.47966,CTN#ANGAZA


In [802]:
df_price.head()

,Nom,type,Cout adopté
0,Adapter Fan,Composant accessoire,1700
1,Panel Fan,Composant accessoire,6500
2,Powerhub 10m of DC cable,Composant accessoire,6500
3,Powerhub AC input cable,Composant accessoire,6500
4,Powerhub Panel 340W,Composant accessoire,140000


In [803]:
df_price.dtypes

Nom            object
type           object
Cout adopté     int64
dtype: object

In [804]:
df_price.columns = ['product_type','type','Cout']

In [805]:
df_compo1 = df_compo.merge(df_price ,on='product_type',how='left') 

In [806]:
df_compo1.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,type,Cout
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,NaN,SKH Controller,pico,Senegal,False,1649441,37707.47966,CTN#ANGAZA,NaN,NaN
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,NaN,SKH Controller,pico,Senegal,False,1308543,37707.47966,CTN#ANGAZA,NaN,NaN
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,NaN,SKH Controller,pico,Senegal,False,1219948,37707.00000,CTN#ANGAZA,NaN,NaN
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,NaN,SKH Controller,pico,Senegal,False,1315845,37707.00000,CTN#ANGAZA,NaN,NaN
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,NaN,SKH Controller,pico,Senegal,False,1214642,37707.47966,CTN#ANGAZA,NaN,NaN


In [821]:
df_system_decom = df_compo1[df_compo1['Cout'].isnull()]

In [822]:
df_system_decom['group'] = 'prinpal'

/tmp/ipykernel_20328/3062457842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_system_decom['group'] = 'prinpal'


In [823]:
col = list(df_system.columns)
col

['product_id',
 'created_at',
 'serial',
 'product',
 'etat',
 'prefix',
 'true_serial',
 'stock_type',
 'region',
 'is_main_system',
 'main_system_id',
 'first_main_system_id',
 'stock_name',
 'systems',
 'product_type',
 'category',
 'country',
 'is_archived',
 'serial_number',
 'cost_price',
 'carton_id',
 'group']

In [824]:
df_remove.head()

,product_type,prix
0,Fan component,8200
1,Hairclipper component,1300
2,Portable light component,0
3,Powerhub 1000 component,153000
4,Powerhub 500 component,153000


In [825]:
df_system_decom = df_system_decom.merge(df_remove,on='product_type',how = 'left')
df_system_decom.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,category,country,is_archived,serial_number,cost_price,carton_id,type,Cout,group,prix
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,pico,Senegal,False,1649441,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,pico,Senegal,False,1308543,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,pico,Senegal,False,1219948,37707.00000,CTN#ANGAZA,NaN,NaN,prinpal,12800.0
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,pico,Senegal,False,1315845,37707.00000,CTN#ANGAZA,NaN,NaN,prinpal,12800.0
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,pico,Senegal,False,1214642,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0


In [826]:
df_system_decom['cost_price2'] = np.where(df_system_decom['cost_price'].notnull(),df_system_decom['cost_price']-df_system_decom['prix'],df_system_decom['cost_price'])
df_system_decom['cost_price3'] = df_system_decom['cost_price']-df_system_decom['prix']
df_system_decom.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,is_archived,serial_number,cost_price,carton_id,type,Cout,group,prix,cost_price2,cost_price3
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,False,1649441,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0,24907.47966,24907.47966
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,False,1308543,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0,24907.47966,24907.47966
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,False,1219948,37707.00000,CTN#ANGAZA,NaN,NaN,prinpal,12800.0,24907.00000,24907.00000
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,False,1315845,37707.00000,CTN#ANGAZA,NaN,NaN,prinpal,12800.0,24907.00000,24907.00000
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,False,1214642,37707.47966,CTN#ANGAZA,NaN,NaN,prinpal,12800.0,24907.47966,24907.47966


In [827]:
print(df_system_decom[df_system_decom['cost_price3']<0]['product_type'].unique())

df_system_decom[df_system_decom['cost_price3']<0]

# df_system_decom.head()

['Powerhub 500 component']


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,is_archived,serial_number,cost_price,carton_id,type,Cout,group,prix,cost_price2,cost_price3
1452,52226,2019-10-16 05:44:44.979517,19062604006,Powerhub 500 component,Reposséder,NaN,19062604006,multi_manager,LAGOS,False,...,True,19062604006,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
5031,52220,2019-10-16 05:44:44.916514,19062604031,Powerhub 500 component,Endommagé,NaN,19062604031,multi_manager,Lagos Warehouse,False,...,False,19062604031,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
9086,52222,2019-10-16 05:44:44.933532,19062604009,Powerhub 500 component,Endommagé,NaN,19062604009,multi_manager,LAGOS,False,...,True,19062604009,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
11117,52224,2019-10-16 05:44:44.961514,19062604010,Powerhub 500 component,Reposséder,NaN,19062604010,multi_manager,LAGOS,False,...,True,19062604010,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
11221,52200,2019-10-16 05:44:44.747753,19062604002,Powerhub 500 component,Endommagé,NaN,19062604002,multi_manager,Lagos Warehouse,False,...,False,19062604002,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
13041,51919,2019-10-08 13:34:54.867012,19062604001,Powerhub 500 component,Neuf,NaN,19062604001,multi_manager,LAGOS,False,...,True,19062604001,122901.62,CTN#PowerHub,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
13107,52191,2019-10-16 05:33:30.316810,19062604028,Powerhub 500 component,Reposséder,NaN,19062604028,multi_manager,Lagos Warehouse,False,...,False,19062604028,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38
13108,52227,2019-10-16 05:44:44.987591,19062604047,Powerhub 500 component,Endommagé,NaN,19062604047,multi_manager,Lagos Warehouse,False,...,False,19062604047,122901.62,CTN#POWERHUB,NaN,NaN,prinpal,153000.0,-30098.38,-30098.38


In [751]:
df_system_decom = df_system_decom[col]

In [752]:
print(df_system_decom.shape)
df_system_decom .head()

(16204, 22)


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,group
0,10688,2017-01-28 12:10:49.000000,1649441,SKH Controller,Reposséder,NaN,1649441,multi_manager,DAKAR,False,...,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1649441,24907.47966,CTN#ANGAZA,prinpal
1,10996,2017-02-08 13:41:16.000000,1308543,SKH Controller,Endommagé,NaN,1308543,multi_manager,FATICK,False,...,Fatick,NaN,SKH Controller,pico,Senegal,False,1308543,24907.47966,CTN#ANGAZA,prinpal
2,11183,2017-02-10 16:41:14.000000,1219948,SKH Controller,Endommagé,NaN,1219948,multi_manager,DAKAR,False,...,Angaza - retraits non repossedes,NaN,SKH Controller,pico,Senegal,False,1219948,24907.00000,CTN#ANGAZA,prinpal
3,11140,2017-02-10 12:17:13.000000,1315845,SKH Controller,Endommagé,NaN,1315845,multi_manager,FATICK,False,...,Fatick,NaN,SKH Controller,pico,Senegal,False,1315845,24907.00000,CTN#ANGAZA,prinpal
4,11192,2017-02-10 17:09:11.000000,1214642,SKH Controller,Endommagé,NaN,1214642,multi_manager,FATICK,False,...,Fatick,NaN,SKH Controller,pico,Senegal,False,1214642,24907.47966,CTN#ANGAZA,prinpal


In [753]:
print(df_system_decom[df_system_decom['cost_price']<0]['product_type'].unique())
df_system_decom[df_system_decom['cost_price']<0]

['Powerhub 500 component']


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,group
1452,52226,2019-10-16 05:44:44.979517,19062604006,Powerhub 500 component,Reposséder,NaN,19062604006,multi_manager,LAGOS,False,...,ZZZ - LGS,NaN,Powerhub 500 component,big SHS,Nigeria,True,19062604006,-30098.38,CTN#POWERHUB,prinpal
5031,52220,2019-10-16 05:44:44.916514,19062604031,Powerhub 500 component,Endommagé,NaN,19062604031,multi_manager,Lagos Warehouse,False,...,LAGOS WAREHOUSE,NaN,Powerhub 500 component,big SHS,Nigeria,False,19062604031,-30098.38,CTN#POWERHUB,prinpal
9086,52222,2019-10-16 05:44:44.933532,19062604009,Powerhub 500 component,Endommagé,NaN,19062604009,multi_manager,LAGOS,False,...,ZZZ - LGS,NaN,Powerhub 500 component,big SHS,Nigeria,True,19062604009,-30098.38,CTN#POWERHUB,prinpal
11117,52224,2019-10-16 05:44:44.961514,19062604010,Powerhub 500 component,Reposséder,NaN,19062604010,multi_manager,LAGOS,False,...,ZZZ - LGS,NaN,Powerhub 500 component,big SHS,Nigeria,True,19062604010,-30098.38,CTN#POWERHUB,prinpal
11221,52200,2019-10-16 05:44:44.747753,19062604002,Powerhub 500 component,Endommagé,NaN,19062604002,multi_manager,Lagos Warehouse,False,...,LAGOS WAREHOUSE,NaN,Powerhub 500 component,big SHS,Nigeria,False,19062604002,-30098.38,CTN#POWERHUB,prinpal
13041,51919,2019-10-08 13:34:54.867012,19062604001,Powerhub 500 component,Neuf,NaN,19062604001,multi_manager,LAGOS,False,...,ZZZ - Sunday Osajiokwu,NaN,Powerhub 500 component,big SHS,Nigeria,True,19062604001,-30098.38,CTN#PowerHub,prinpal
13107,52191,2019-10-16 05:33:30.316810,19062604028,Powerhub 500 component,Reposséder,NaN,19062604028,multi_manager,Lagos Warehouse,False,...,LAGOS WAREHOUSE,NaN,Powerhub 500 component,big SHS,Nigeria,False,19062604028,-30098.38,CTN#POWERHUB,prinpal
13108,52227,2019-10-16 05:44:44.987591,19062604047,Powerhub 500 component,Endommagé,NaN,19062604047,multi_manager,Lagos Warehouse,False,...,LAGOS WAREHOUSE,NaN,Powerhub 500 component,big SHS,Nigeria,False,19062604047,-30098.38,CTN#POWERHUB,prinpal


In [717]:
df_system_decom[df_system_decom['cost_price']<0].groupby('product_type')['true_serial'].count()

product_type
Fan component               117
Oolu TV Screen 24            13
Oolu TV Screen 32             8
Powerhub 1000 component       5
Powerhub 1280 component       8
Powerhub 500 component        8
SKH Controller             7350
Name: true_serial, dtype: int64

In [684]:
df_system_decom.groupby('product_type')['true_serial'].count()

product_type
Fan component                257
Hairclipper USB cable          3
Hairclipper component          8
Oolu TV Screen 24             14
Oolu TV Screen 32             35
Oolu TV Screen 43              7
Portable light component      17
Powerbank component            4
Powerhub 1000 component        5
Powerhub 1280 component        8
Powerhub 500 component         8
Radio component              138
SKH Controller              7350
WowSolar Controller         8343
Name: true_serial, dtype: int64

In [706]:
print(df_system_decom.shape)
df_system_decom.isnull().sum()

(16197, 22)


product_id                  0
created_at                  0
serial                      0
product                     0
etat                        0
prefix                  15783
true_serial                 0
stock_type                  0
region                      0
is_main_system              0
main_system_id          16197
first_main_system_id    16065
stock_name                  0
systems                 16197
product_type                0
category                  277
country                     0
is_archived                 0
serial_number             132
cost_price                500
carton_id                2062
group                       0
dtype: int64

In [707]:
df_composant= df_compo1[df_compo1['Cout'].notnull()]
print(df_composant.shape)
df_composant.head()

(55504, 23)


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,type,Cout
1,284323,2021-02-14 19:19:18.298639,5328554C2,SKH Lamp,Endommagé,NaN,5328554C2,multi_manager,THIES,False,...,NaN,SKH Lamp,pico,Senegal,False,NaN,NaN,NaN,Composant accessoire,3800.0
3,341502,2021-02-14 19:29:48.337128,80111106C2,WowSolar Lamp 140lm,Reposséder,NaN,80111106C2,personal,MATAM,False,...,NaN,WowSolar Lamp 140lm,pico,Senegal,False,NaN,NaN,NaN,Composant accessoire,3000.0
4,857374,2021-03-29 17:33:35.069380,50108760C1,WowSolar Remote control,Reposséder,NaN,50108760C1,multi_manager,DAKAR,False,...,NaN,WowSolar Remote control,NaN,Senegal,False,NaN,NaN,NaN,Composant accessoire,4000.0
7,813907,2021-02-19 23:03:01.425848,50041980C6,WowSolar Extension cable,Neuf,NaN,50041980C6,personal,KAYES,False,...,NaN,WowSolar Extension cable,pico,Mali,False,NaN,NaN,NaN,Composant accessoire,650.0
10,719505,2021-02-19 22:17:14.113020,20014668C10,Satellite Cable 12m incl. connectors,Reposséder,NaN,20014668C10,multi_manager,DAKAR,False,...,NaN,Satellite Cable 12m incl. connectors,SHS+TV,Senegal,False,NaN,NaN,NaN,Composant accessoire,1500.0


In [708]:
df_composant['group'] = 'composant'
df_composant['cost_price'] = df_composant['Cout']

/tmp/ipykernel_20328/3131649247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_composant['group'] = 'composant'
/tmp/ipykernel_20328/3131649247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_composant['cost_price'] = df_composant['Cout']


In [709]:
df_composant = df_composant[col]

In [710]:
df_composant.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,group
1,284323,2021-02-14 19:19:18.298639,5328554C2,SKH Lamp,Endommagé,NaN,5328554C2,multi_manager,THIES,False,...,Thies,NaN,SKH Lamp,pico,Senegal,False,NaN,3800.0,NaN,composant
3,341502,2021-02-14 19:29:48.337128,80111106C2,WowSolar Lamp 140lm,Reposséder,NaN,80111106C2,personal,MATAM,False,...,SEYDOU ABDOULAYE DIALLO - MATAMPOLY33,NaN,WowSolar Lamp 140lm,pico,Senegal,False,NaN,3000.0,NaN,composant
4,857374,2021-03-29 17:33:35.069380,50108760C1,WowSolar Remote control,Reposséder,NaN,50108760C1,multi_manager,DAKAR,False,...,Angaza - retraits non repossedes,NaN,WowSolar Remote control,NaN,Senegal,False,NaN,4000.0,NaN,composant
7,813907,2021-02-19 23:03:01.425848,50041980C6,WowSolar Extension cable,Neuf,NaN,50041980C6,personal,KAYES,False,...,Bakary Kamissoko - kita04,NaN,WowSolar Extension cable,pico,Mali,False,NaN,650.0,NaN,composant
10,719505,2021-02-19 22:17:14.113020,20014668C10,Satellite Cable 12m incl. connectors,Reposséder,NaN,20014668C10,multi_manager,DAKAR,False,...,Angaza - retraits non repossedes,NaN,Satellite Cable 12m incl. connectors,SHS+TV,Senegal,False,NaN,1500.0,NaN,composant


In [711]:
print(df_composant.shape)
df_composant.isnull().sum()

(55504, 22)


product_id                  0
created_at                  0
serial                      0
product                     0
etat                        0
prefix                  53075
true_serial                 0
stock_type                  0
region                      0
is_main_system              0
main_system_id          55492
first_main_system_id    26275
stock_name                  0
systems                 55504
product_type                0
category                 1079
country                     0
is_archived                 0
serial_number           54787
cost_price                  0
carton_id               55377
group                       0
dtype: int64

In [712]:
data = pd.concat([df_system,df_system_decom,df_composant],axis=0)

In [713]:
print(data.shape)
data.head()

(76990, 22)


,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,group
0,47311,2019-04-04 09:08:45.256129,10032417,WowSolar Controller,Neuf,NaN,10032417,personal,TILLABERI,True,...,halidou abdou - halidou.abdou,WowSolar60,WowSolar60,pico,Niger,False,10032417,34089.00,CTN#A000003218,system
2,869417,2021-04-22 15:44:12.888708,745847974005507119,Powerbank component,Neuf,745.0,847974005507119,personal,KOGI,True,...,Juliet Gbakaan - juliet_gbakaan,Sherpa Powerbank,Sherpa Powerbank,accessories,Nigeria,False,847974005507119,NaN,NaN,system
12,950560,2022-03-14 19:01:47.039286,4544521430153,Oolu TV Screen 43,Neuf,454.0,4521430153,multi_manager,BAMAKO,True,...,BAMAKO,Oolu SolarTV43,Oolu SolarTV43,SHS+TV,Mali,False,4521430153,256311.00,NaN,system
16,924106,2021-09-21 16:38:26.697096,4521320277,Oolu TV Screen 32,Neuf,NaN,4521320277,multi_manager,ZIGUINCHOR,True,...,Ziguinchor,Oolu SolarTV32,Oolu SolarTV32,SHS+TV,Senegal,False,4521320277,208690.00,NaN,system
22,56909,2020-01-22 15:17:39.543211,L1901909000789,Portable light component,Neuf,NaN,L1901909000789,multi_manager,Lagos Warehouse,True,...,LAGOS WAREHOUSE,L190 Portable light + phone charger,L190 Portable light + phone charger,NaN,Nigeria,False,L1901909000789,6844.78,CTN#OMNIVOLTAIC,system


In [693]:
data.columns

Index(['product_id', 'created_at', 'serial', 'product', 'etat', 'prefix',
       'true_serial', 'stock_type', 'region', 'is_main_system',
       'main_system_id', 'first_main_system_id', 'stock_name', 'systems',
       'product_type', 'category', 'country', 'is_archived', 'serial_number',
       'cost_price', 'carton_id', 'group'],
      dtype='object')

In [694]:
data.dtypes

product_id                int64
created_at               object
serial                   object
product                  object
etat                     object
prefix                  float64
true_serial              object
stock_type               object
region                   object
is_main_system             bool
main_system_id          float64
first_main_system_id    float64
stock_name               object
systems                  object
product_type             object
category                 object
country                  object
is_archived                bool
serial_number            object
cost_price              float64
carton_id                object
group                    object
dtype: object

In [695]:
# data['cost_price'] = data['cost_price'].astype(float)
# data[data['cost_price']=='3\u202f800,00']
# data.dtypes

In [696]:
value_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | data['is_archived']==True)
loss_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | ~data['stock_name'].str.lower().str.startswith('zzz'))
reimburse_condition = (data['etat'].isin(['Endommagé','waste','canceled']) | ~data['stock_name'].str.lower().str.startswith('yyy'))


data['valeur'] = np.where(value_condition,0,data['cost_price'])
data['loss'] = np.where(loss_condition,0,data['cost_price'])
data['reimburse'] = np.where(reimburse_condition,0,data['cost_price'])


data.head()

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,category,country,is_archived,serial_number,cost_price,carton_id,group,valeur,loss,reimburse
0,47311,2019-04-04 09:08:45.256129,10032417,WowSolar Controller,Neuf,NaN,10032417,personal,TILLABERI,True,...,pico,Niger,False,10032417,34089.00,CTN#A000003218,system,34089.00,0.0,0.0
2,869417,2021-04-22 15:44:12.888708,745847974005507119,Powerbank component,Neuf,745.0,847974005507119,personal,KOGI,True,...,accessories,Nigeria,False,847974005507119,NaN,NaN,system,NaN,0.0,0.0
12,950560,2022-03-14 19:01:47.039286,4544521430153,Oolu TV Screen 43,Neuf,454.0,4521430153,multi_manager,BAMAKO,True,...,SHS+TV,Mali,False,4521430153,256311.00,NaN,system,256311.00,0.0,0.0
16,924106,2021-09-21 16:38:26.697096,4521320277,Oolu TV Screen 32,Neuf,NaN,4521320277,multi_manager,ZIGUINCHOR,True,...,SHS+TV,Senegal,False,4521320277,208690.00,NaN,system,208690.00,0.0,0.0
22,56909,2020-01-22 15:17:39.543211,L1901909000789,Portable light component,Neuf,NaN,L1901909000789,multi_manager,Lagos Warehouse,True,...,NaN,Nigeria,False,L1901909000789,6844.78,CTN#OMNIVOLTAIC,system,6844.78,0.0,0.0


In [600]:
data.isnull().sum()

product_id                  0
created_at                  0
serial                      0
product                     0
etat                        0
prefix                  72201
true_serial                 0
stock_type                  0
region                      0
is_main_system              0
main_system_id          76978
first_main_system_id    47629
stock_name                  0
systems                 71701
product_type                0
category                 2234
country                     0
is_archived                 0
serial_number           54942
cost_price               2340
carton_id               60241
group                       0
valeur                   1758
loss                      122
reimburse                  80
dtype: int64

In [601]:
data.dtypes

product_id                int64
created_at               object
serial                   object
product                  object
etat                     object
prefix                  float64
true_serial              object
stock_type               object
region                   object
is_main_system             bool
main_system_id          float64
first_main_system_id    float64
stock_name               object
systems                  object
product_type             object
category                 object
country                  object
is_archived                bool
serial_number            object
cost_price              float64
carton_id                object
group                    object
valeur                  float64
loss                    float64
reimburse               float64
dtype: object

In [602]:
print(f"{data['valeur'].sum()}")

501332765.14523995


In [603]:
print(f"{data['loss'].sum()}")

17708426.05626


In [604]:
print(f"{data['reimburse'].sum()}")

5806745.8366


In [714]:
data['category'].unique()

array(['pico', 'accessories', 'SHS+TV', nan, 'big SHS'], dtype=object)

In [715]:
data

,product_id,created_at,serial,product,etat,prefix,true_serial,stock_type,region,is_main_system,...,stock_name,systems,product_type,category,country,is_archived,serial_number,cost_price,carton_id,group
0,47311,2019-04-04 09:08:45.256129,10032417,WowSolar Controller,Neuf,NaN,10032417,personal,TILLABERI,True,...,halidou abdou - halidou.abdou,WowSolar60,WowSolar60,pico,Niger,False,10032417,34089.00,CTN#A000003218,system
2,869417,2021-04-22 15:44:12.888708,745847974005507119,Powerbank component,Neuf,745.0,847974005507119,personal,KOGI,True,...,Juliet Gbakaan - juliet_gbakaan,Sherpa Powerbank,Sherpa Powerbank,accessories,Nigeria,False,847974005507119,NaN,NaN,system
12,950560,2022-03-14 19:01:47.039286,4544521430153,Oolu TV Screen 43,Neuf,454.0,4521430153,multi_manager,BAMAKO,True,...,BAMAKO,Oolu SolarTV43,Oolu SolarTV43,SHS+TV,Mali,False,4521430153,256311.00,NaN,system
16,924106,2021-09-21 16:38:26.697096,4521320277,Oolu TV Screen 32,Neuf,NaN,4521320277,multi_manager,ZIGUINCHOR,True,...,Ziguinchor,Oolu SolarTV32,Oolu SolarTV32,SHS+TV,Senegal,False,4521320277,208690.00,NaN,system
22,56909,2020-01-22 15:17:39.543211,L1901909000789,Portable light component,Neuf,NaN,L1901909000789,multi_manager,Lagos Warehouse,True,...,LAGOS WAREHOUSE,L190 Portable light + phone charger,L190 Portable light + phone charger,NaN,Nigeria,False,L1901909000789,6844.78,CTN#OMNIVOLTAIC,system
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71694,900935,2021-05-14 13:34:36.682353,3376FB000080,Fan Lithium Battery,Neuf,337.0,6FB000080,multi_manager,LAGOS,False,...,XXX - Sold Component,NaN,Fan Lithium Battery,NaN,Nigeria,True,6FB000080,3600.00,NaN,composant
71695,179866,2021-02-14 19:00:17.734469,40031469C8,WowSolar Wall mount,Reposséder,NaN,40031469C8,personal,MATAM,False,...,ADAMA SALAMATA BA - matampoly18,NaN,WowSolar Wall mount,pico,Senegal,False,NaN,400.00,NaN,composant
71697,85054,2021-02-14 18:39:30.296477,6919120C5,SKH USB cable,Reposséder,NaN,6919120C5,multi_manager,DAKAR,False,...,Angaza - retraits non repossedes,NaN,SKH USB cable,pico,Senegal,False,NaN,1000.00,NaN,composant
71698,259531,2021-02-14 19:15:11.461211,80104140C5,WowSolar Lamp 140lm,Neuf,NaN,80104140C5,personal,SIKASSO,False,...,Hamaye Makanguile - hamaye.makanguile@oolusola...,NaN,WowSolar Lamp 140lm,pico,Mali,False,NaN,3000.00,NaN,composant


In [718]:
data[data['cost_price']<0].groupby('product_type')['true_serial'].count()

product_type
Fan component               117
Oolu TV Screen 24            13
Oolu TV Screen 32             8
Powerhub 1000 component       5
Powerhub 1280 component       8
Powerhub 500 component        8
SKH Controller             7350
Name: true_serial, dtype: int64